In [2]:
import pandas as pd
try:
    data = pd.read_csv("../data/titanic_proc.csv", index_col="PassengerId")
except:
    data = pd.read_csv("https://raw.githubusercontent.com/Argentan/DMA_LAB2/master/data/titanic_proc.csv", index_col="PassengerId")
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,NumFam,C,Q,S,Age_nul
PassengerId,,,,,,,,,,,,,,
1,0,3,0,22.0,1,0,0,7.2500,0,1,0,0,1,0
2,1,1,1,38.0,1,0,1,71.2833,1,1,1,0,0,0
3,1,3,1,26.0,0,0,2,7.9250,0,0,0,0,1,0
4,1,1,1,35.0,1,0,3,53.1000,2,1,0,0,1,0
5,0,3,0,35.0,0,0,4,8.0500,0,0,0,0,1,0


In [3]:
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, KFold, ParameterGrid

OSError: dlopen(/Users/emlanza/anaconda3/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/emlanza/anaconda3/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=1)


In [ ]:
folds = [train.index[idx] for _, idx in KFold(5, shuffle=True, random_state=1).split(train)]


In [ ]:
candidatos = [
    {"algoritmo": LogisticRegression, "params": {"C": [0.1, 1]}},
    {"algoritmo": LGBMClassifier, "params": {}}
]

In [ ]:
valid_probs = []
test_probs = []
for candidato in candidatos:
    for params in ParameterGrid(candidato["params"]):
        test_fold_probs = []
        valid_fold_probs = []
        name = candidato["algoritmo"].__name__ + ";" + ";".join([f"{k}_{v}" for k, v in params.items()])
        for valid_idx in folds:
            X_valid = train.loc[valid_idx].drop("Survived", axis=1)
            # y_valid = train.loc[valid_idx, "Survived"]

            X_train = train.drop(valid_idx).drop("Survived", axis=1)
            y_train = train.loc[X_train.index, "Survived"]

            model = candidato["algoritmo"](**params)
            model.fit(X_train, y_train)
            test_fold_probs.append(model.predict_proba(test.drop("Survived", axis=1))[:, -1])
            valid_fold_probs.append(model.predict_proba(X_valid)[:, -1])
        break
    break